In [1]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2023-06-16 18:04:49.097958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Model Parameters
batch_size = 16
learning_rate = 0.001
EPOCHS = 25

In [3]:
# Set the directory paths for the benign and malignant images
benign_dir = 'Dataset/Benign Masses'
malignant_dir = 'Dataset/Malignant Masses'

# Target size for resizing
target_size = (224, 224)

# Lists to store the images and labels
images = []
labels = []

# Function to resize and process the images
def process_image(image_path, label):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, target_size)
    images.append(resized_image)
    labels.append(label)

# Processing the benign images
for filename in os.listdir(benign_dir):
    image_path = os.path.join(benign_dir, filename)
    process_image(image_path, label='benign')

# Processing the malignant images
for filename in os.listdir(malignant_dir):
    image_path = os.path.join(malignant_dir, filename)
    process_image(image_path, label='malignant')

In [4]:
#Converting lists to numpy arrays
images = np.array(images)/255.0
labels = np.array(labels)

In [5]:
#Encoding labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]


In [6]:
#One-hot-encoding our labels
labels = to_categorical(labels)
print(labels)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 

In [7]:
# Assuming X contains your feature data and y contains your target labels
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Data Augmentation

In [8]:
#Data augmentation
trainAug = ImageDataGenerator(rotation_range=15, fill_mode = "nearest")

In [9]:
#VGG16 Model introduction
baseModel = VGG16(weights = "imagenet", include_top = False, input_tensor=Input(shape = (224,224,3)))

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16

# Create an instance of the VGG16 model
model = VGG16()

# Call the summary() method on the model instance
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Building the last layer of the model

In [11]:
#Connecting the basemodel (VGG16) to our output layer
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (4,4)) (headModel)
headModel = Flatten(name = "flatten") (headModel)
headModel = Dense(64, activation="relu") (headModel)
headModel = Dropout(0.5) (headModel)
headModel = Dense(2, activation="softmax") (headModel)

In [12]:
#Combine the models
model = Model(inputs = baseModel.input, outputs = headModel)

In [13]:
#Freezing the baseModel layers
for layer in baseModel.layers:
    layer.trainable = False

In [14]:
#Compiling The model using Adam optimizer
#learning_rate = 0.001
#decay_rate = learning_rate / epochs
#opt = Adam(learning_rate, decay_rate)
#model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

#model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
#Train the head model
#early_stop = EarlyStopping(patience=10,min_delta= 0.001, restore_best_weights=True)

#---------------------------

# Compile the model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stop = EarlyStopping(patience=5, restore_best_weights=True)


# Fit the model
H = model.fit(trainAug.flow(X_train,y_train, 
            batch_size= batch_size),
            steps_per_epoch = len(X_train) // batch_size,
            validation_data = (X_test, y_test),
            validation_steps = len(X_test) // batch_size,
            epochs = EPOCHS,
            callbacks = [early_stop]
            )

# Evaluate the model

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/25


2023-06-16 18:05:14.050494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


21/21 [==============================] - 75s 4s/step - loss: 0.7437 - accuracy: 0.5422 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 2/25
21/21 [==============================] - 75s 4s/step - loss: 0.6989 - accuracy: 0.5422 - val_loss: 0.7134 - val_accuracy: 0.4828
Epoch 3/25
21/21 [==============================] - 75s 4s/step - loss: 0.6974 - accuracy: 0.5301 - val_loss: 0.7196 - val_accuracy: 0.4828
Epoch 4/25
21/21 [==============================] - 75s 4s/step - loss: 0.6904 - accuracy: 0.5181 - val_loss: 0.6936 - val_accuracy: 0.6379
Epoch 5/25
21/21 [==============================] - 76s 4s/step - loss: 0.6958 - accuracy: 0.5241 - val_loss: 0.7003 - val_accuracy: 0.5000
Epoch 6/25
21/21 [==============================] - 74s 4s/step - loss: 0.6779 - accuracy: 0.5783 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 7/25
 1/21 [>.............................] - ETA: 44s - loss: 0.7938 - accuracy: 0.5000

KeyboardInterrupt: 